> 任务

1. sklearn转换器和预估器
2. KNN算法
3. 模型选择与调优
4. 朴素贝叶斯算法
5. 决策树
6. 随机森林

## 3.1.sklearn转换器和估计器（estimator）
### 3.1.1.转换器 -- 特征工程的父类
之前做特征工程的步骤
1. 实例化（实例化的是一个转换器类(Transformer)）
2. 调用fit_transform(对于文档建立分类词矩阵，不能同时调用)

我们把特征工程的接口称之为转换器，其中转换器有如下几种类型：
1. fit_transform
2. fit(): 计算每一列的平均值、标准差
3. transform()：进行公式的调用，与最终的转换

### 3.1.2.估计器 --  sklearn机器学习算法的父类
在sklearn中,估计器(estimation)是一个重要的角色,是一类实现了算法的API

+ (1)用于分类的估计值:
    + sklearn.neighbors k- 邻近算法
    + sklearn.naivate_bayes 贝叶斯
    + sklearn.linnear_mordel.LogisticRegression 逻辑回归
    + sklearn.tree 决策树与随机森林
+ (2)用于回归的估计器:
    + sklearn.linear_model.LinearRegression 线性回归
    + sklearn.linear_model.Ridge 岭回归
+ (3)用于无监督学习
    + sklearn.cluster.KMeans 聚类 

估计器工作流程

![估计器流程](../../img/估计器工作流程.png)

> 步骤

1. 实例化一个estimator
2. estimator.fit(x_train,y_train) 计算 
3. 调用完毕,模型生成
4. 模型评估
   1. 直接比对真实值和预测  
        y_predict = estimator.predict()
        y_test == y_predict
   2. 计算准确率
        accuracy = estimator.score(x_test,y_test)

## 3.2.K-邻近算法

> 目标

+ 说明K-近邻算法的距离公式
+ 说明K-近邻算法的超参数K值以及取值问题
+ 说明K-近邻算法的优缺点
+ 应用KNeighborsClassifier实现分类
+ 了解分类算法的评估评估标准准确率

> 应用

+ 鸢尾花数据预测
+ Facebook签到位置预测

> 核心思想

    K Nearest Neighbor 算法又叫KNN算法,根据你的"邻居"推断出你的类别  
    物以类聚，人以群分  
    KNN 中的 K 指的是「K 个」邻居，K=3 就是通过离的最近的 3 个样本，来判断新数据的类别。

> 定义
 
    如果一个样本在特征空间中的k个值最相似(即特征空间中最邻近)的样本中的大多数属于某一个类别,则该样本也属于这个类别
    KNN算法最早是由Cover和Hart提出的一个分类算法
  
> 距离公式

两个样本的距离可以通过如下公式计算,又叫欧式距离  
![欧氏距离](../../img/欧氏距离.png)

其他距离公式:曼哈顿距离\绝对值距离\明可斯夫基距离


> K-近邻算法API

k值取得值过小,容易收到异常点的影响
k值取得值过大,样本容易收到不均衡的影响
![近邻值API](../../img/近邻值API.png)

> 案例鸢尾花种类预测

![鸢尾花](../../img/鸢尾花数据集.png)

> 步骤

1. 获取数据
2. 数据集划分
3. 特征工程(标准化)
4. 机器学习训练
5. 模型评估

> K-近邻总结


In [14]:
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.neighbors import KNeighborsClassifier

def knn_iris():
    """
    用KNN算法对鸢尾花进行分类
    :return:
    """
    # 1）获取数据
    iris = load_iris()

    # 2）划分数据集
    x_train, x_test, y_train, y_test = train_test_split(iris.data, iris.target, random_state=22)

    # 3）特征工程：标准化
    transfer = StandardScaler()
    x_train = transfer.fit_transform(x_train)
    x_test = transfer.transform(x_test)

    # 4）KNN算法预估器
    estimator = KNeighborsClassifier(n_neighbors=33)
    estimator.fit(x_train, y_train)

    # 5）模型评估
    # 方法1：直接比对真实值和预测值
    y_predict = estimator.predict(x_test)
    print("y_predict:\n", y_predict)
    print("直接比对真实值和预测值:\n", y_test == y_predict)

    # 方法2：计算准确率
    score = estimator.score(x_test, y_test)
    print("准确率为：\n", score)

    return None

if __name__ == "__main__":
    knn_iris()

y_predict:
 [0 2 1 2 1 2 1 1 1 0 2 1 2 2 0 2 1 1 1 1 0 2 0 1 2 0 2 2 2 2 0 0 1 1 1 0 0
 0]
直接比对真实值和预测值:
 [ True  True  True  True  True False  True False  True  True  True  True
  True  True  True  True  True  True False  True  True  True  True  True
  True  True  True  True  True  True  True  True  True  True  True  True
  True  True]
准确率为：
 0.9210526315789473


### 3.3.模型选择和调优
#### 3.3.1.交叉验证(cross validation)

交叉验证:将拿到的训练数据,分为训练集和验证集.如下图所示,将数据分成4份,其中一份作为验证集,然后经过4次的测试,每次都更换不同的验证集,即得到4组模型的结果,取平均值作为最终结果.又称4折交叉验证.

> 分析
> 
我们知道数据分为训练集和测试集,但是为了让训练集得到模型结果更加准确.做以下处理
+ 训练集: 训练集+验证集
+ 测试集: 测试集
  ![训练集和验证集](../../img/训练集和验证集.png)

> 目的

为了让被评估的模型更加准确可信

#### 3.3.2.超参数搜索-网格搜索(Grid Search)
通常情况下,有很多参数是需要手动指定的(如K-邻近算法中的K值),这种叫做超参数,但是手动过程复杂,所以需要对模型预设几种超参数组合.每组超参数都采用交叉验证来进行评估,最后选出最优参数组合建立模型  
![超参数](../../img/超参数.png)  
  
#### 3.3.3.模型选择与调优API
![模型选择与调优API](../../img/模型选择与调优api.png)


In [ ]:
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import GridSearchCV


def knn_iris_gscv():
    """
    用KNN算法对鸢尾花进行分类,添加网格搜索和交叉验证
    :return:
    """
    # 1）获取数据
    iris = load_iris()

    # 2）划分数据集
    x_train, x_test, y_train, y_test = train_test_split(iris.data, iris.target, random_state=22)

    # 3）特征工程：标准化
    transfer = StandardScaler()
    x_train = transfer.fit_transform(x_train)
    x_test = transfer.transform(x_test)

    # 4）KNN算法预估器，metric选项可以修改距离的计算公式，如minkowski
    estimator = KNeighborsClassifier()

    # 加入网格搜索和交叉验证
    # 参数准备
    param_dict = {'n_neighbors':[1,3,5,7,9,11]}
    # cv设置交叉验证的次数
    estimator = GridSearchCV(estimator,param_grid=param_dict,cv=10)
    estimator.fit(x_train, y_train)

    # 5）模型评估
    # 方法1：直接比对真实值和预测值
    y_predict = estimator.predict(x_test)
    print("y_predict:\n", y_predict)
    print("直接比对真实值和预测值:\n", y_test == y_predict)

    # 方法2：计算准确率
    score = estimator.score(x_test, y_test)
    print("准确率为：\n", score)

    # 最佳参数: best_params_
    print("最佳参数\n",estimator.best_params_)
    # 最佳结果: best_score_
    print("最佳结果",estimator.best_score_)
    # 最佳估计器: best_estimator_
    print("最佳估计器",estimator.best_estimator_)
    # 交叉验证结果: cv_results_
    print("交叉验证结果",estimator.cv_results_)

    return None

if __name__ == "__main__":
    knn_iris_gscv()

## 3.4 预测FaceBook签到位置
> File Description

![facebook数据说明](../../img/Facebook数据说明.png)
![facebook中文数据集](../../img/facebook中文数据集.png)

> 官网 [https://www.kaggle.com/navoshta/grid-knn/data](https://www.kaggle.com/navoshta/grid-knn/data)